In [1]:
import pandas as pd
import fasttext
from sklearn.metrics import f1_score

In [2]:
# 转换为FastText需要的格式
train_df = pd.read_csv('./data/train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].to_csv('./data/train_fasttext.csv', index=None, header=None, sep='\t')

In [30]:
model = fasttext.train_supervised('./data/train_fasttext.csv',  lr=1.0, wordNgrams=3, 
                                  verbose=2, minCount=1, epoch=50, loss="hs")

In [31]:
val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df['text']]
print(f1_score(train_df['label'].values.astype(str), val_pred, average='macro'))

0.9999321893792349


In [32]:
test_df = pd.read_csv('./data/test_a.csv', sep='\t')
test_pred = [model.predict(x)[0][0].split('__')[-1] for x in test_df['text']]

In [33]:
test_pred = pd.Series(test_pred, name='label', dtype='int8')
print(test_pred[:10])
test_pred.to_csv('./data/submit_ft.csv', index=False)

0    1
1    2
2    8
3    5
4    0
5    4
6    2
7    1
8    6
9    4
Name: label, dtype: int8


/home/ygao/anaconda3/lib/python3.7/site-packages/numpy/core/numeric.py:2327: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [29]:
model.labels
model.get_dimension()

100

In [34]:
model.save_model('./data/fasttext.h5')

In [35]:
model.t

0.0001